# Исследование надежности заемщиков.

**Цель проекта:** выяснить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.  
**Входные данные:** файл от банка со статистикой о платёжеспособности клиентов.  
**План работ:**
1. Изучить данные 
1. Предобработать данные: удалить пропуски, исправить аномалии, категоризоровать 
1. Проверить гипотезы о зависимсоти погашения кредита в срок от разных факторов

## Обзор данных
<a name="1._Обзор_данных"></a>

Смотрим содержимое датасета и описание данных:

In [1]:
#импортируем библиотеки
import pandas as pd
from IPython.display import display
#читаем данные из файла
try:
    data = pd.read_csv('data.csv')
except:
    data = pd.read_csv('datasets/data.csv')
#выводим содержимое и общую информацию
display(data) 
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных
<a name="2._Предобработка_данных"></a>

###  Заполнение пропусков
<a name="2.1_Заполнение_пропусков"></a>

В столбцах `days_employed`  и `total_income` есть пропуски. Это можно увидеть из вывода метода `info()` в предыдущем разделе.  
Проверим какую долю составляют пропущенные значения в каждом из столбцов с пропусками:

In [2]:
#вычисляем доли
days_employed_null_part = data['days_employed'].isna().sum() / len(data)
total_income_null_part = data['total_income'].isna().sum() / len(data)
#выводим отформатированно в %
print(f'Доля пропусков в столбце days_employed: {days_employed_null_part:.0%}')
print(f'Доля пропусков в столбце total_income: {total_income_null_part:.0%}')

Доля пропусков в столбце days_employed: 10%
Доля пропусков в столбце total_income: 10%


Вероятно, в данных столбцах появились пропуски, так как не все клиенты предоставили банку данные о трудовом стаже и ежемесячном доходе.

Заполним пропуски медианным значением.  
*Именно **медианным**, а не средним арифметическим, так как таким образом статистика не будет искажена из-за выбросов (аномалий) в данных.*

In [3]:
#заполняем пропуски
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
#проверяем результат 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Проверка данных на аномалии и исправления
<a name="2.2_Проверка_данных_на_аномалии_и_исправления"></a>

Проверим все столбцы на наличие аномальных значений:

In [4]:
#предварительно переведем дни трудового стажа в года и внесем данные в колонку years_employed
data['years_employed'] = data['days_employed'] / 365
#приведем годы к целому числу
data['years_employed'] = data['years_employed'].astype('int') 
#удалим колонку 'days_employed'
data = data.drop(['days_employed'], axis= 1)

#проходим по всем столбцам и выводим уникальные значения и их количество
for column in data.columns:
    print('Столбец', column)
    print(data[column].value_counts())
    print()
 

Столбец children
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Столбец dob_years
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Столбец education
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268

Обнаружены следующие аномалии: 
1. `children` 
    1. Есть отрицательные значения. 47 клиентов с -1 ребенком. Скорее всего опечатка и в этих строках должен быть 1 ребенок.
    1. 76 клиентов с 20 детьми - не очень похоже на правду. Скорее всего опечатка и в этих строках должно быть 2 ребенка.
1. `years_employed` 
    1. Есть отрицательные значения. Опечатка или ошибка при выгрузке данных. Заменим на положительные
    1. Есть значения > 900. Найдем все значения где трудовой стаж больше чем возраст клиента и заменим эти значения на среднее по медиане.
1. `dob_years`
    1. Есть значения 0. Несовершеннолетние не могут быть заемщиками. Опечатка или ошибка при выгрузке данных. Но таких значений < 1%, поэтому оставим их без изменений
    


Обработаем столбцы с аномалиями:

In [5]:
#В столбце children заменим -1 на 1
data.loc[data['children'] == -1, 'children'] = 1
#В столбце children заменим 20 на 2
data.loc[data['children'] == 20, 'children'] = 2
#Проверим результат
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [6]:
#В столбце years_employed заменим отрицательные значения на положительные: возьмем значение по модулю
data['years_employed'] =  data['years_employed'].abs()
#В столбце years_employed найдем все значения, где стаж больше возраста и заменим их на медианные
data.loc[data['years_employed'] > data['dob_years'], 'years_employed'] = data['years_employed'].median()

#Проверим результат
data['years_employed'].value_counts()

4     4859
3     3717
1     2018
2     1864
0     1827
5     1019
6      961
7      808
8      746
9      570
10     455
11     376
12     345
13     279
14     251
15     205
16     133
19     124
17     117
18      97
20      90
21      88
22      80
23      65
24      53
25      51
26      50
27      46
30      34
29      28
32      28
31      27
28      25
33      17
34      12
35      12
36      10
37       8
38       7
39       6
40       4
43       3
41       3
42       2
44       2
48       1
45       1
50       1
Name: years_employed, dtype: int64

###  Изменение типов данных
<a name="Изменение_типов_данных"></a>

Заменим в столбце `total_income` тип с вещенственного на целочисленный

In [7]:
#заменяем тип
data['total_income'] = data['total_income'].astype('int') 
#проверяем результат
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   dob_years         21525 non-null  int64 
 2   education         21525 non-null  object
 3   education_id      21525 non-null  int64 
 4   family_status     21525 non-null  object
 5   family_status_id  21525 non-null  int64 
 6   gender            21525 non-null  object
 7   income_type       21525 non-null  object
 8   debt              21525 non-null  int64 
 9   total_income      21525 non-null  int64 
 10  purpose           21525 non-null  object
 11  years_employed    21525 non-null  int64 
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Удаление дубликатов
<a name="Удаление_дубликатов"></a>

#### Явные дубликаты
<a name="Явные_дубликаты"></a>

In [8]:
#проверяем есть ли яввные дубликаты и сколько их
data.duplicated().sum()
#удаляем явные дубликаты
data = data.drop_duplicates().reset_index(drop= True)

#### Неявные дубликаты
<a name="Неявные_дубликаты"></a>

Проверим неявные дубликаты в столбцах в текстовым содержимым `family_status`, `education`, `income_type`, `purpose`

In [9]:
print(data['family_status'].value_counts())
print()
print(data['education'].value_counts())
print()
print(data['income_type'].value_counts())
print()
print(data['purpose'].value_counts())

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу    

* В столбцах `family_status` и `income_type` неявных дубликатов не обнаружено.   
* В столбце `education` найдены неявные дубликаты. Они отличаются регистром. Приведем все значения к нижнему регистру. 
* В столбце `purpose` также найдены неявные дубликаты. Они отличаются написанием. Можно выявить следующие причины: свадьба, образование, автомобиль,  недвижимсоть. **Но поскольку в одном из следующих заданий требуется сделать категоризацию по данному столбцу, что фактически ознаечет обход подобных дубликатов, в этом задании оставим этот столбец как есть.** 

In [10]:
#приведем все значения в столбце education к нижнему регистру
data['education'] = data['education'].str.lower()
#проверим результат
print(data['education'].value_counts())

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


#### Возможные причины возникновения дубликатов

* Явные - дублирование строк при формировании файла-выгрузки
* Неявные - клиенты указывали образование и причину для кредита вручную, а не выбирали из списка, поэтому появились разные варианты написания

###  Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма
<a name="Формирование_дополнительных_датафреймов_словарей,_декомпозиция_исходного_датафрейма"></a>

In [11]:
#Создадим датафрейм education_dict
education_dict = data[['education_id', 'education']]
#Удалим дубликаты
education_dict = education_dict.drop_duplicates().reset_index(drop= True)
#Проверим результат
print(education_dict.value_counts())

#Удалим столбец 'education' из датафрейма data
data = data.drop(['education'], axis= 1) 
#Проверим результат
data.columns

education_id  education          
0             высшее                 1
1             среднее                1
2             неоконченное высшее    1
3             начальное              1
4             ученая степень         1
dtype: int64


Index(['children', 'dob_years', 'education_id', 'family_status',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose', 'years_employed'],
      dtype='object')

In [12]:
#Создадим датафрейм family_status_dict
family_status_dict = data[['family_status_id', 'family_status']]
#Удалим дубликаты
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop= True)
#Проверим результат
print(family_status_dict.value_counts())

#Удалим столбец 'family_status' из датафрейма data
data = data.drop(['family_status'], axis= 1) 
#Проверим результат
data.columns

family_status_id  family_status        
0                 женат / замужем          1
1                 гражданский брак         1
2                 вдовец / вдова           1
3                 в разводе                1
4                 Не женат / не замужем    1
dtype: int64


Index(['children', 'dob_years', 'education_id', 'family_status_id', 'gender',
       'income_type', 'debt', 'total_income', 'purpose', 'years_employed'],
      dtype='object')

###  Категоризация дохода
<a name="Категоризация_дохода"></a>

In [13]:
#создадим функцию, которая принимает значение и в зависимсоти от него возвращает название категории
def set_total_income_category (income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        else: return 'A'
    except:  
        return 'No category'

#применим функцию к столбцу total_income. Результат выполнения поместим в столбец total_income_category
data['total_income_category'] = data['total_income'].apply(set_total_income_category)
#проверим резульат
print(data['total_income_category'].value_counts())

C    16033
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64


### Категоризация целей кредита
<a name="Категоризация_целей_кредита"></a>

In [14]:
#создадим функцию, которая принимает значение и в зависимсоти  от него возвращает название категории
def set_purpose_category (purpose):
    try: 
        if 'свад' in purpose:
            return 'проведение свадьбы'
        elif 'образ' in purpose:
            return 'получение образования'
        elif 'автомобил' in purpose:
            return 'операции с автомобилем'
        elif 'недвиж' in purpose or 'жиль'in purpose:
            return 'операции с недвижимостью'
        else: return 'другое'
    except:  
        return('другое')

#применим функцию к столбцу purpose. Результат выполнения поместим в столбец purpose_category
data['purpose_category'] = data['purpose'].apply(set_purpose_category)
#проверим резульат
print(data['purpose_category'].value_counts())

операции с недвижимостью    10814
операции с автомобилем       4308
получение образования        4014
проведение свадьбы           2335
Name: purpose_category, dtype: int64


## Ответы на вопросы
<a name="Ответы_на_вопросы"></a>

### Зависимость между количеством детей и возвратом кредита в срок
<a name="Зависимость_между_количеством_детей_и_возвратом_кредита_в_срок"></a>

Для определения наличия данной зависимости сгруппируем данные по кол-ву детей и вычислим процент задолжностей по кредиту в каждой группе.  
Имеет смысл сравнивать именно относительные величины, так как клиентов 5 детьми меньше чем клиентов без детей. 

In [15]:
#составим сводную таблицу. Строки - кол-во детей, столбец - сумма кредитов с задолжностью
children_pivot = data.pivot_table(index= 'children', values ='debt',  aggfunc='sum' )
#добавим столбец no_debt - сумма кредитов без задолжностей
children_pivot['no_debt'] = data.groupby('children')['debt'].count() - children_pivot['debt']
#добавим столбец debt_percent - процент задолжностей от общего кол-ва кредитов в каждой группе
children_pivot['debt_percent'] = children_pivot['debt'] /  (children_pivot['debt'] + children_pivot['no_debt']) * 100
#выводим данные отсортированными по столбцу debt_percent
display(children_pivot.sort_values(by= 'debt_percent'))


,debt,no_debt,debt_percent
children,,,
5,0,9,0.000000
0,1063,13044,7.535266
3,27,303,8.181818
1,445,4411,9.163921
2,202,1926,9.492481
4,4,37,9.756098


**Вывод:** Меньше всего задолжностей у семей с 5 детьми, а больше всего - с 4. Явной корреляции между количеством детей и возвратом в срок не прослеживается.  
Следует заметить, что клиентов с 5 детьми в датасете всего 9, это составлявляет меньше тысячной процента от общего числа клиентов. Данные по данной категории клиентов могут быть нерелевантными.  
Можно сделать вывод, что **зависимости нет.**

### Зависимость между семейным положением и возвратом кредита в срок
<a name="Зависимость_между_семейным_положением_и_возвратом_кредита_в_срок"></a>

Для определения наличия данной зависимости сгруппируем данные по семейному положению и вычислим процент задолжностей по кредиту в каждой группе.  
Так же, как и в случае выше, будем сравнивать относительные показатели. 

In [16]:
#объединим датасеты data и family_status_dict, чтобы в выводе отображались названия семейных положений, а не их id, поскольку так результат будет выглядеть информативнее.
family = data.merge(family_status_dict, on= 'family_status_id', how='left')
# cоставим сводную таблицу. Строки - семейное положение, столбец - сумма кредитов с задолжностью
family_pivot = family.pivot_table(index= 'family_status', values ='debt',  aggfunc='sum' )
#добавим столбец no_debt - сумма кредитов без задолжностей
family_pivot['no_debt'] = family.groupby('family_status')['debt'].count() - family_pivot['debt']
#добавим столбец debt_percent - процент задолжностей от общего кол-ва кредитов в каждой группе
family_pivot['debt_percent'] = family_pivot['debt'] /  (family_pivot['debt'] + family_pivot['no_debt']) * 100

#выводим данные отсортированными по столбцу debt_percent
display(family_pivot.sort_values(by= 'debt_percent'))


,debt,no_debt,debt_percent
family_status,,,
вдовец / вдова,63,896,6.569343
в разводе,85,1110,7.112971
женат / замужем,931,11413,7.542126
гражданский брак,388,3775,9.320202
Не женат / не замужем,274,2536,9.750890


**Вывод:** наибольший процент задолжностей у клиентов не состоящих в браке, наименьший - у вдовцов/ вдов. При этом разница между наибольшим и наименьшим значением составляет ~3%.  
Семейное положение - это категориальная переменная, то есть нельзя сравнивать значения между собой. Также явного лидера и антилидера по показателю возвратов в срок не наблюдается.   
Можно сказать, что **зависимости нет.**

### Зависимость между уровнем дохода и возвратом кредита в срок
<a name="Зависимость_между_уровнем_дохода_и_возвратом_кредита_в_срок"></a>

Для определения наличия данной зависимости сгруппируем данные по категории дохода и вычислим процент задолжностей по кредиту в каждой группе.  
Так же, как и в случае выше, будем сравнивать относительные показатели. 

In [17]:
#составим сводную таблицу. Строки - категория дохода, столбец - сумма кредитов с задолжностью
total_income_pivot = data.pivot_table(index= 'total_income_category', values ='debt',  aggfunc='sum' )
#добавим столбец no_debt - сумма кредитов без задолжностей
total_income_pivot['no_debt'] = data.groupby('total_income_category')['debt'].count() - total_income_pivot['debt']
#добавим столбец debt_percent - процент задолжностей от общего кол-ва кредитов в каждой группе
total_income_pivot['debt_percent'] = total_income_pivot['debt'] /  (total_income_pivot['debt'] + total_income_pivot['no_debt']) * 100

#выводим данные отсортированными по столбцу debt_percent
display(total_income_pivot.sort_values(by= 'debt_percent'))

,debt,no_debt,debt_percent
total_income_category,,,
D,21,329,6.000000
B,356,4685,7.062091
A,2,23,8.000000
C,1360,14673,8.482505
E,2,20,9.090909


**Вывод:** наибольший процент задолжностей у клиентов категории Е, это минимальный уровень дохода. Однако наименьший - у клиентов категории D, это вторая с конца категория по уровню дохода. Внутри последовательности таже не прослеживается корреляции между уровнем дохода и процентом задолжностей. Разница между наибольшим и наименьшим значением также составляет ~3%.  
Также можно сделать вывод, что **зависимости нет.**

### Влияние разных целей кредита на возврат в срок
<a name="Влияние_разных_целей_кредита_на_возврат_в_срок"></a>

Для определения наличия данной зависимости сгруппируем данные по категории цели кредита и вычислим процент задолжностей по кредиту в каждой группе.  
Так же, как и в случае выше, будем сравнивать относительные показатели.

In [18]:
#составим сводную таблицу. Строки - категория цели, столбец - сумма кредитов с задолжностью
purpose_pivot = data.pivot_table(index= 'purpose_category', values ='debt',  aggfunc='sum' )
#добавим столбец no_debt - сумма кредитов без задолжностей
purpose_pivot['no_debt'] = data.groupby('purpose_category')['debt'].count() - purpose_pivot['debt']
#добавим столбец debt_percent - процент задолжностей от общего кол-ва кредитов в каждой группе
purpose_pivot['debt_percent'] = purpose_pivot['debt'] /  (purpose_pivot['debt'] + purpose_pivot['no_debt']) * 100

#выводим данные отсортированными по столбцу debt_percent
display(purpose_pivot.sort_values(by= 'debt_percent'))

,debt,no_debt,debt_percent
purpose_category,,,
операции с недвижимостью,782,10032,7.231367
проведение свадьбы,186,2149,7.965739
получение образования,370,3644,9.217738
операции с автомобилем,403,3905,9.354689


**Вывод:** наибольший процент задолжностей наблюдается по кредитам на операции с автомобилем, а наименьший - с недвижимостью. Но разница между наибольшим и наименьшим значением снова небольшая: ~2%. 
Цель кредита - категориальная переменная, то есть нельзя сравнить ее значения между собой. При этом явного лидера и антилидера по проценту задолжностей не выявлено.   
Можно сделать вывод, что цель кредита практически **не влияет** на возврат в срок.

### Зависимость между уровнем образования и возвратом кредита в срок
<a name="Зависимость_между_уровнем_образования_и_возвратом_кредита_в_срок"></a>

*В изначальной постановке задачи не было такого требования, но, как аналитик, я считаю нужным проверить и эту гипотезу, так как мы располагаем необходимыми данными.*

Для определения наличия данной зависимости сгруппируем данные по уровню образования и вычислим процент задолжностей по кредиту в каждой группе.  
Так же, как и в случае выше, будем сравнивать относительные показатели.

In [19]:
#объединим датасеты data и education_dict, чтобы в выводе отображались названия уровней образования, а не их id, поскольку так результат будет выглядеть информативнее.
education = data.merge(education_dict, on= 'education_id', how='left')

# cоставим сводную таблицу. Строки - семейное положение, столбец - сумма кредитов с задолжностью
education_pivot = education.pivot_table(index= 'education', values ='debt',  aggfunc='sum' )
#добавим столбец no_debt - сумма кредитов без задолжностей
education_pivot['no_debt'] = education.groupby('education')['debt'].count() - education_pivot['debt']
#добавим столбец debt_percent - процент задолжностей от общего кол-ва кредитов в каждой группе
education_pivot['debt_percent'] = education_pivot['debt'] /  (education_pivot['debt'] + education_pivot['no_debt']) * 100

#выводим данные отсортированными по столбцу debt_percent
display(education_pivot.sort_values(by= 'debt_percent'))


,debt,no_debt,debt_percent
education,,,
ученая степень,0,6,0.000000
высшее,278,4973,5.294230
среднее,1364,13824,8.980774
неоконченное высшее,68,676,9.139785
начальное,31,251,10.992908


**Вывод:** наибольший процент задолжностей у клиентов с начальным образованием, наименьший - с ученой степенью. Почти все остальные значения также располагаются по убыванию. Исключение: среднее и неоконченное высшее, но  разница между ними <1%. 
Разница между наибольшим и наименьшим значением ~11%. Это более заметный результат, чем в предыдущих гипотезах.   
Можно сделать вывод, что **клиенты с наиболее высоким уровнем образования лучше выплачивают кредиты в срок.** 

## Выводы
<a name="Выводы"></a>

В ходе исследования были проверены 5 гипотез о влиянии разных факторов на возврат кредита в срок: 

1. Количество детей - не влияет
1. Семейное положение - не влияет
1. Уровень дохода - не влияет
1. Цель кредита - не влияет    
1. Уровень образования (дополнительно) - есть зависимость: чем выше уровень образования, тем меньше задолжностей. 


Отвечая на изначальный запрос клиента, влияет ли семейное положение и количество детей на возврат в срок: **нет, не влияет.**  
Однако, при скоринге рекомендуется обратить внимание на уровень образования клиента.